In [3]:
source("../../../MIRACLE-reproducibility/preprocess/utils.R")
base_dir <- "../../../MIRACLE-reproducibility/data/raw/dcm_hcm/"
count_path <- pj(base_dir)
counts <- Read10X(count_path)
obj <- gen_rna(counts)
obj
meta <- read.csv(paste0(base_dir, 'DCM_HCM_MetaData_V1.txt'), sep='\t')
rownames(meta) <- meta[['NAME']]
meta <- meta[2:dim(meta)[1], ]

Attaching SeuratObject

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



In [26]:
obj[['donor_id']] <- meta['donor_id']
obj[['cell_type__ontology_label']] <- meta['cell_type__ontology_label']
obj[['disease__ontology_label']] <- meta['disease__ontology_label']
obj[['cell_type_leiden06']] <- meta['cell_type_leiden06']
obj[['cell_type__ontology_label']] <- meta['cell_type__ontology_label']
obj[['cell_type_leiden06']] <- substr(obj@meta.data[['cell_type_leiden06']], 4, 40)
label <- obj@meta.data[['cell_type_leiden06']]
for(i in c(unique(unique(obj@meta.data[['cell_type_leiden06']])))){
    if(i=="Fibroblast_II"){
        label[label==i] = 'Fibroblast'
    }else if(i=="Fibroblast_I"){
        label[label==i] = 'Fibroblast'
    }else if(i=="Cardiomyocyte_I"){
        label[label==i] = 'Cardiomyocyte'
    }else if(i=="Cardiomyocyte_II"){
        label[label==i] = 'Cardiomyocyte'
    }else if(i=="Cardiomyocyte_III"){
        label[label==i] = 'Cardiomyocyte'
    }else if(i=="Pericyte_I"){
        label[label==i] = 'Pericyte'
    }else if(i=="Pericyte_II"){
        label[label==i] = 'Pericyte'
    }else if(i=="Endothelial_I"){
        label[label==i] = 'Endothelial'
    }else if(i=="Endothelial_II"){
        label[label==i] = 'Endothelial'
    }else if(i=="Endothelial_III"){
        label[label==i] = 'Endothelial'
    }
    else if(i=="Proliferating_macrophage"){
        label[label==i] = 'Macrophage'
    }
}
obj[['l1']] <- obj@meta.data[['cell_type_leiden06']]
obj[['l2']] <- label
rna_list <- SplitObject(obj, split.by = "donor_id")


In [91]:
output_dir_list <- list()
p <- list()
for(i in c(1:42)){
    mkdir(paste0(base_dir, unique(rna_list[[i]][['donor_id']])[1, 1], '/label_seurat/'))
    rna_list[[i]] <- gen_rna(rna_list[[i]][['rna']]@counts)
    p[[i]] <- VlnPlot(rna_list[[i]], c("nFeature_rna", "nCount_rna"),
        pt.size = 0.001, ncol = 3, log = F) + NoLegend()
}
mkdir(paste0(base_dir, '/vlnplot_before'))
for ( i in c(1:42)){
    ggsave(paste0(base_dir, '/vlnplot_before/',unique(obj[['donor_id']])[i,1], '.png'), p[[i]])
}

In [111]:
mkdir(paste0(base_dir, '/vlnplot_processed'))
for(i in c(1:42)){
    rna <- rna_list[[i]]
    rna <- subset(rna, subset =
        nFeature_rna > 400 & nFeature_rna < 4000 & 
        nCount_rna > 500 & nCount_rna < 13000
    )
    # p <- VlnPlot(rna, c("nFeature_rna", "nCount_rna"),
    #         pt.size = 0.001, ncol = 3) + NoLegend()
    # rna
    # ggsave(paste0(base_dir, '/vlnplot_processed/',unique(obj[['donor_id']])[i,1], '.png'), p)
    # preprocess(output_dir_list[[i]], rna = rna)
    write.csv(obj@meta.data[colnames(rna), ]['l1'], paste0(base_dir, unique(obj@meta.data[colnames(rna), ][['donor_id']])[1], '/label_seurat/l1.csv'))
    write.csv(obj@meta.data[colnames(rna), ]['l2'], paste0(base_dir, unique(obj@meta.data[colnames(rna), ][['donor_id']])[1], '/label_seurat/l2.csv'))
    write.csv(obj@meta.data[colnames(rna), ]['disease__ontology_label'], paste0(base_dir, unique(obj@meta.data[colnames(rna), ][['donor_id']])[1], '/label_seurat/disease.csv'))
}